In [12]:
import cv2
import numpy as np
from keras.datasets import mnist
from keras.layers import Dense, Flatten, MaxPooling2D, Dropout
from keras.layers.convolutional import Conv2D
from keras.models import Sequential
from keras.utils import to_categorical
import matplotlib.pyplot as plt

In [13]:
def dataLoading():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.reshape(60000, 28, 28, 1)
    X_test = X_test.reshape(10000, 28, 28, 1)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    return (X_train,y_train,X_test, y_test)

In [14]:
def modelBuilding():
    model = Sequential()

## Declare the layers
    layer_1 = Conv2D(64, kernel_size=3, activation='relu', input_shape=(28, 28, 1))
    layer_2 = MaxPooling2D(pool_size=2)
    layer_3 = Conv2D(32, kernel_size=3, activation='relu')
    layer_4 = MaxPooling2D(pool_size=2)
    layer_5 = Dropout(0.5)
    layer_6 = Flatten()
    layer_7 = Dense(128, activation="relu")
    layer_8 = Dropout(0.5)
    layer_9 = Dense(10, activation='softmax')

    ## Add the layers to the model
    model.add(layer_1)
    model.add(layer_2)
    model.add(layer_3)
    model.add(layer_4)
    model.add(layer_5)
    model.add(layer_6)
    model.add(layer_7)
    model.add(layer_8)
    model.add(layer_9)
    return model


In [15]:
def train():
    model  = modelBuilding()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    (X_train,y_train,X_test,y_test) = dataLoading()
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)
    return model

In [16]:
def captureImage():
    image = cv2.imread('./5.jpg')
    grey = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)

    ret, thresh = cv2.threshold(grey.copy(), 75, 255, cv2.THRESH_BINARY_INV)
    contours,_= cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    preprocessed_digits = []

    for c in contours:
        x,y,w,h = cv2.boundingRect(c)

        # Creating a rectangle around the digit in the original image (for displaying the digits fetched via contours)
        cv2.rectangle(image, (x,y), (x+w, y+h), color=(0, 255, 0), thickness=2)

        # Cropping out the digit from the image corresponding to the current contours in the for loop
        digit = thresh[y:y+h, x:x+w]

        # Resizing that digit to (18, 18)
        resized_digit = cv2.resize(digit, (18,18))

        # Padding the digit with 5 pixels of black color (zeros) in each side to finally produce the image of (28, 28)
        padded_digit = np.pad(resized_digit, ((5,5),(5,5)), "constant", constant_values=0)

        # Adding the preprocessed digit to the list of preprocessed digits
        preprocessed_digits.append(padded_digit)

    print("\n\n\n----------------Contoured Image--------------------")
    plt.imshow(image, cmap="gray")
    plt.show()

    inp = np.array(preprocessed_digits)
    return preprocessed_digits

In [17]:
def modelTrain():
    preprocessed_digits = captureImage()
    model = train()
    for digit in preprocessed_digits:
        prediction = model.predict(digit.reshape(1, 28, 28, 1))   
        print ("\n\n---------------------------------------\n\n")
        print ("=========PREDICTION============ \n\n")
        plt.imshow(digit.reshape(28, 28), cmap="gray")
        plt.show()
        print("\n\nFinal Output: {}".format(np.argmax(prediction)))
        print ("\nPrediction (Softmax) from the neural network:\n\n {}".format(prediction))
        hard_maxed_prediction = np.zeros(prediction.shape)
        hard_maxed_prediction[0][np.argmax(prediction)] = 1
        print ("\n\nHard-maxed form of the prediction: \n\n {}".format(hard_maxed_prediction))
        print ("\n\n---------------------------------------\n\n")